In [2]:
# 模块导入和参数设置
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

# 超参数
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # 最优选择动作百分比
GAMMA = 0.9                 # 奖励递减参数
TARGET_REPLACE_ITER = 100   # Q 现实网络的更新频率
MEMORY_CAPACITY = 2000      # 记忆库大小
env = gym.make('CartPole-v0')   # 立杆子游戏
env = env.unwrapped
N_ACTIONS = env.action_space.n  # 杆子能做的动作
N_STATES = env.observation_space.shape[0]   # 杆子能获取的环境信息数

# 神经网络
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 10)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.out = nn.Linear(10, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)   # initialization

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value
        
# DQN 体系
class DQN(object):
    # 建立 target net 和 eval net 还有 memory
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()

        self.learn_step_counter = 0     # 用于 target 更新计时
        self.memory_counter = 0         # 记忆库记数
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # 初始化记忆库
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)    # torch 的优化器
        self.loss_func = nn.MSELoss()   # 误差公式
    
    # 根据环境观测值选择动作的机制
    def choose_action(self, x):
        x = torch.FloatTensor(x).unsqueeze(0)
        # 这里只输入一个 sample
        if np.random.uniform() < EPSILON:   # 选最优动作
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].numpy()[0]     # return the argmax
        else:   # 选随机动作
            action = np.random.randint(0, N_ACTIONS)
        return action
    
    # 存储记忆
    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # 如果记忆库满了, 就覆盖老数据
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1
    
    # target 网络更新
    # 学习记忆库中的记忆
    def learn(self):
        # target net 参数更新
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # 抽取记忆库中的批数据
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # 针对做过的动作b_a, 来选 q_eval 的值, (q_eval 原本有所有动作的值)
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # q_next 不进行反向传递误差, 所以 detach
        q_target = b_r + GAMMA * q_next.max(1)[0]   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        # 计算, 更新 eval net
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()



In [3]:
# 训练
dqn = DQN() # 定义 DQN 系统

for i_episode in range(400):
    s = env.reset()
    while True:
        # env.render()    # 显示实验动画
        a = dqn.choose_action(s)

        # 选动作, 得到环境反馈
        s_, r, done, info = env.step(a)

        # 修改 reward, 使 DQN 快速学习
        x, x_dot, theta, theta_dot = s_
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2

        # 存记忆
        dqn.store_transition(s, a, r, s_)

        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn() # 记忆库满了就进行学习

        if done:    # 如果回合结束, 进入下回合
            break

        s = s_
env.close()

/Users/v_hansongyang/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [4]:
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = dqn.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

In [5]:
b_s

tensor([[-0.0409, -0.0454, -0.0077,  0.0128],
        [ 0.0682,  0.9718, -0.0480, -1.4803],
        [ 0.0047,  0.0387,  0.0159, -0.0373],
        [ 0.0424,  0.3752, -0.0374, -0.6508],
        [-0.0054, -0.4420,  0.1223,  0.7480],
        [-0.0439,  0.1578, -0.0138, -0.4093],
        [ 0.0434,  0.6272,  0.0022, -0.8389],
        [ 0.0113,  0.8258, -0.0861, -1.3072],
        [ 0.0646,  0.4533, -0.1547, -0.9123],
        [-0.0124,  0.8231, -0.0572, -1.2279],
        [-0.0110,  0.1519,  0.0181, -0.2485],
        [-0.0826, -1.1858,  0.1219,  1.8377],
        [-0.1020,  0.2146,  0.2077,  0.0973],
        [ 0.1666,  0.7949, -0.1855, -1.2920],
        [-0.0224, -0.7554, -0.0062,  1.1067],
        [ 0.0577,  0.9607, -0.0789, -1.5399],
        [-0.0403, -0.0303,  0.0128,  0.0244],
        [ 0.0098,  0.7449, -0.0390, -1.1890],
        [-0.0275,  0.2399,  0.0108, -0.3246],
        [ 0.0250, -0.7669, -0.0362,  1.0422],
        [ 0.0027,  0.4352, -0.0263, -0.6271],
        [-0.0423,  0.7928, -0.0288

In [6]:
dqn.eval_net(b_s)

tensor([[1.9158, 1.9213],
        [1.5634, 1.5654],
        [1.8751, 1.8814],
        [1.7009, 1.7032],
        [1.3147, 1.2893],
        [1.8582, 1.8630],
        [1.8873, 1.8910],
        [1.4193, 1.4208],
        [1.1027, 1.1004],
        [1.5797, 1.5826],
        [1.8844, 1.8927],
        [1.0860, 1.0818],
        [0.9287, 0.8911],
        [0.8975, 0.8923],
        [1.8152, 1.8176],
        [1.4085, 1.4097],
        [1.8572, 1.8640],
        [1.6514, 1.6547],
        [1.8993, 1.9082],
        [1.8861, 1.8874],
        [1.7824, 1.7857],
        [1.6790, 1.6840],
        [1.4261, 1.4247],
        [0.9477, 0.9417],
        [1.5277, 1.5258],
        [1.6328, 1.6209],
        [1.5797, 1.5826],
        [1.7023, 1.6900],
        [1.6621, 1.6649],
        [0.9747, 0.9718],
        [1.6383, 1.6403],
        [1.0698, 1.0389]], grad_fn=<AddmmBackward>)

In [7]:
b_a

tensor([[0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1]])

In [8]:
dqn.eval_net(b_s).gather(1, b_a)

tensor([[1.9158],
        [1.5654],
        [1.8814],
        [1.7032],
        [1.2893],
        [1.8630],
        [1.8910],
        [1.4208],
        [1.1004],
        [1.5826],
        [1.8927],
        [1.0860],
        [0.8911],
        [0.8975],
        [1.8152],
        [1.4097],
        [1.8640],
        [1.6547],
        [1.9082],
        [1.8861],
        [1.7857],
        [1.6840],
        [1.4261],
        [0.9477],
        [1.5277],
        [1.6209],
        [1.5826],
        [1.6900],
        [1.6621],
        [0.9718],
        [1.6403],
        [1.0389]], grad_fn=<GatherBackward>)